# Mathematical model kinetics

In [1]:
#import warnings
#warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [47]:
import pandas as pd
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 10000)
import numpy as np
import yaml

from vessel_manoeuvring_models.visualization.plot import track_plot, plot
from phd.visualization.plot_ship import track_plots
from vessel_manoeuvring_models.prime_system import PrimeSystem
from phd.visualization.plot_prediction import plot_total_force, plot_force_components, predict, plot_compare_model_forces, plot_parameter_contributions
from phd.visualization.bokeh_plotting import create_tab
from vessel_manoeuvring_models.symbols import *
import matplotlib.pyplot as plt
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify, remove_functions, prime
from vessel_manoeuvring_models.parameters import df_parameters
p = df_parameters["symbol"]
import statsmodels.api as sm
import sympy as sp
from sympy import Eq, symbols, latex
import paper
from paper import save_fig, save_eq

In [72]:
#%reload_kedro
ship='wPCC'
ship_data = catalog.load(f"{ship}.ship_data")
model_loaders = catalog.load(f"{ship}.models_VCT")
models_rudder_VCT_loaders = catalog.load(f"{ship}.models_rudder_VCT")

[01/09/24 07:34:25] INFO     Loading data from 'wPCC.ship_data' (YAMLDataSet)...                ]8;id=113885;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=211678;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'wPCC.models_VCT' (PartitionedDataset)...        ]8;id=771573;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=600765;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    WARNING  /home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/par ]8;id=533150;file:///home/maa/anaconda3/envs/phd/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=699061;file:///home/maa/anaconda3/envs/phd/lib/python3.10/warnings.py#109\109]8;;\
                             titioned_dataset.py:200: KedroDeprecationWarning: 'PartitionedDataset'                
                             has been moved to `kedro-datasets` and will be removed in Kedro                       
                             0.19.0.                                                                               
                               warnings.warn(                                                                      
                                                                                                                   

                    INFO     Loading data from 'wPCC.models_rudder_VCT' (PartitionedDataset)... ]8;id=573730;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=536805;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

In [7]:
model_loaders

{'semiempirical_covered': <bound method AbstractDataset.load of <phd.extras.datasets.model_dataset.ModularVesselSimulatorDataSet object at 0x7f4d68087040>>}

In [73]:
models_rudder_VCT_loaders

{'Abkowitz': <bound method AbstractDataset.load of <phd.extras.datasets.model_dataset.ModularVesselSimulatorDataSet object at 0x7f4d6236cc40>>}

In [8]:
model = model_loaders['semiempirical_covered']()

In [75]:
model_abkowitz = models_rudder_VCT_loaders['Abkowitz']()

In [9]:
model.X_eq

Eq(m*(\dot{u} - r**2*x_G - r*v), X_RHI + X_{\dot{u}}*\dot{u} + X_H(u, v, r, delta) + X_P(u, v, r, rev) + X_R(u, v, r, delta, thrust) + X_W(C_x3, C_x2, C_x0, C_x4, C_x5, aws, C_x1, A_XV, awa, rho_A))

In [22]:
eq_X = Eq(model.X_eq.lhs,X_D)
eq_Y = Eq(model.Y_eq.lhs,Y_D)
eq_N = Eq(model.N_eq.lhs,N_D)

save_eq(remove_functions(eq_X), file_name='X')
save_eq(remove_functions(eq_Y), file_name='Y')
save_eq(remove_functions(eq_N), file_name='N')

'mathematical_model_kinetics.X'

Eq(m*(\dot{u} - r**2*x_G - r*v), X_D)

''

'mathematical_model_kinetics.Y'

Eq(m*(\dot{r}*x_G + \dot{v} + r*u), Y_D)

''

'mathematical_model_kinetics.N'

Eq(I_z*\dot{r} + m*x_G*(\dot{v} + r*u), N_D)

''

In [71]:
X_D_eq = remove_functions(model.X_D_eq).subs([
    ("X_W",0),
    ("X_RHI",0),
])
Y_D_eq = remove_functions(model.Y_D_eq).subs("Y_W",0)
N_D_eq = remove_functions(model.N_D_eq).subs("N_W",0)

save_eq(remove_functions(X_D_eq), file_name='X_D')
save_eq(remove_functions(Y_D_eq), file_name='Y_D')
save_eq(remove_functions(N_D_eq), file_name='N_D')

'mathematical_model_kinetics.X_D'

Eq(X_D, X_H + X_P + X_R)

''

'mathematical_model_kinetics.Y_D'

Eq(Y_D, Y_H + Y_P + Y_R + Y_RHI)

''

'mathematical_model_kinetics.N_D'

Eq(N_D, N_H + N_P + N_R + N_RHI)

''

In [26]:
model.show_subsystems()

hull: PrimeEquationSubSystem
propellers: PropellersSimpleSystem
rudder_port: SemiempiricalRudderSystemCovered
rudder_stbd: SemiempiricalRudderSystemCovered
rudders: Rudders
rudder_hull_interaction: RudderHullInteractionSystem
wind_force: DummyWindForceSystem


## Hull

In [51]:
for name, eq in model.subsystems['hull'].equations.items():
    save_eq(prime(eq), file_name=name)

'mathematical_model_kinetics.X_H'

Eq({X_H'}, {X_{0}'} + {X_{rr}'}*{r'}**2 + {X_{u}'}*{u'} + {X_{vr}'}*{r'}*{v'} + {X_{vv}'}*{v'}**2)

''

'mathematical_model_kinetics.Y_H'

Eq({Y_H'}, {Y_{0}'} + {Y_{rrr}'}*{r'}**3 + {Y_{r}'}*{r'} + {Y_{vrr}'}*{r'}**2*{v'} + {Y_{vvr}'}*{r'}*{v'}**2 + {Y_{vvv}'}*{v'}**3 + {Y_{v}'}*{v'})

''

'mathematical_model_kinetics.N_H'

Eq({N_H'}, {N_{0}'} + {N_{rrr}'}*{r'}**3 + {N_{r}'}*{r'} + {N_{vrr}'}*{r'}**2*{v'} + {N_{vvr}'}*{r'}*{v'}**2 + {N_{vvv}'}*{v'}**3 + {N_{v}'}*{v'})

''

## Propellers

In [116]:
for name, eq in model.subsystems['propellers'].equations.items():
    if isinstance(model.subsystems['propellers'], vessel_manoeuvring_models.models.subsystem.PrimeEquationSubSystem):
        save_eq(prime(eq), file_name=name)
    else:
        save_eq(eq, file_name=name)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Rudder hull interaction

In [70]:
for name, eq in model.subsystems['rudder_hull_interaction'].equations.items():
    if isinstance(model.subsystems['rudder_hull_interaction'], vessel_manoeuvring_models.models.subsystem.PrimeEquationSubSystem):
        save_eq(prime(eq), file_name=name)
    else:
        save_eq(eq, file_name=name)

'mathematical_model_kinetics.X_RHI'

Eq(X_RHI, 0)

''

'mathematical_model_kinetics.Y_RHI'

Eq(Y_RHI, Y_R*a_H)

''

'mathematical_model_kinetics.N_RHI'

Eq(N_RHI, N_R*x_H)

''

## Pure mathematical rudder model

In [76]:
model_abkowitz.show_subsystems()

hull: PrimeEquationSubSystem
propellers: PropellersSimpleSystem
rudders: AbkowitzRudderSystem
rudder_hull_interaction: RudderHullInteractionDummySystem
wind_force: DummyWindForceSystem


In [108]:
for name, eq in model_abkowitz.subsystems['rudders'].equations.items():
    
    file_name = f"{name}_math"
    
    if isinstance(model_abkowitz.subsystems['rudders'], vessel_manoeuvring_models.models.subsystem.PrimeEquationSubSystem):
        save_eq(prime(eq), file_name=file_name)
    else:
        save_eq(eq, file_name=file_name)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Semi-empirical rudder model

In [110]:
rudder = model.subsystems['rudder_port']

In [150]:
rudder.equations['X_R_port']

Eq(X_R_port, -D_R_port*cos(alpha_f_port) + L_R_port*sin(alpha_f_port))

In [129]:
keys = list(rudder.equations.keys())
keys.reverse()

In [155]:
for name in keys:
    eq = rudder.equations[name]
    name_clean = name.replace('_port','')
    file_name = f"{name_clean}_semiempirical"    
    save_eq(eq, file_name=file_name, subs={'port':''})

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

'\\begin{equation}\n    \\label{eq:c_semiempirical}\n    \\input{equations/mathematical_model_kinetics.c_semiempirical}\n\\end{equation}'

In [142]:
s_latex = ""
for name in keys:
    eq = rudder.equations[name]
    name_clean = name.replace('_port','')
    file_name = f"{name_clean}_semiempirical"
    
    eq_latex = fr"""
\begin{{equation}}
\label{{eq:{file_name}}}
\input{{equations/mathematical_model_kinetics.{file_name}}}
\end{{equation}}
%"""
    s_latex+=eq_latex

print(s_latex)


\begin{equation}
\label{eq:N_R_semiempirical}
\input{equations/mathematical_model_kinetics.N_R_semiempirical}
\end{equation}
%
\begin{equation}
\label{eq:Y_R_semiempirical}
\input{equations/mathematical_model_kinetics.Y_R_semiempirical}
\end{equation}
%
\begin{equation}
\label{eq:X_R_semiempirical}
\input{equations/mathematical_model_kinetics.X_R_semiempirical}
\end{equation}
%
\begin{equation}
\label{eq:L_R_semiempirical}
\input{equations/mathematical_model_kinetics.L_R_semiempirical}
\end{equation}
%
\begin{equation}
\label{eq:L_R_U_semiempirical}
\input{equations/mathematical_model_kinetics.L_R_U_semiempirical}
\end{equation}
%
\begin{equation}
\label{eq:L_R_C_semiempirical}
\input{equations/mathematical_model_kinetics.L_R_C_semiempirical}
\end{equation}
%
\begin{equation}
\label{eq:lambda_R_semiempirical}
\input{equations/mathematical_model_kinetics.lambda_R_semiempirical}
\end{equation}
%
\begin{equation}
\label{eq:f_semiempirical}
\input{equations/mathematical_model_kinetics.f_s